<a target="_blank" href="https://colab.research.google.com/github/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Uncomment the next snippet if you are in colab or need to install mhar library!

In [ ]:
#!pip install mhar

This tutorial will show how to use `mhar` for sampling full dimensional polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs. If you want to se a tutorial on how to sample non-full dimensional polytopes see [tutorial](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb)

In [1]:
import torch

Before starting let's check if you have an avaialble gpu device or not.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [3]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$

For fully dimensional polytopes we need to use the class `Polytope` in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-hypercube = \{x \in R^n || x \in [-1,1]^n \} $  

Which we can represent in matrix restrictions:  
$ Ix \leq 1$  
$ -Ix \leq 1$  
Where $I$ is the identity matrix of dimension $n \times n$ 


We will use this restrictions to define the polytope as:  
$A^Ix = [I | -I]x \leq 1 = b^I$

#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope.

In [4]:
n = 3 # Dimension
dtype = torch.float32 # Precision 
A_I = torch.cat((torch.eye(n), torch.eye(n) * -1.0), dim=0).to(dtype) # Inequality Matrix
b_I = torch.ones(2 * n, dtype=dtype).view(-1, 1)  # Inequality restriction vector      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}')

Inequality Matrix A^I 
 tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]]) 

Inequality Vector b^I 
 tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


Now lets create a `Polytope` object to represent the polytope.

In [5]:
from mhar.polytope import Polytope
hypercube = Polytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [6]:
hypercube

Numeric Precision (dtype) torch.float32
Device: cuda
A_in: torch.Size([6, 3]) 
b_in: torch.Size([6, 1])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [7]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [8]:
x0 = ChebyshevCenter(polytope=hypercube, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device=device, # device used cpu or cuda
                    solver_precision=np.float32 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [9]:
x0

tensor([[-0.],
        [-0.],
        [-0.]], device='cuda:0')

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [10]:
x0 = torch.cat([x0, 
            torch.tensor([[.5], [.5], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[-0.0000, 0.5000],
        [-0.0000, 0.5000],
        [-0.0000, 0.5000]], device='cuda:0')

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [11]:
from mhar.walk import walk
X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=3**3, 
        device= None, 
        seed=None,
        verbosity=2
)

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  3   mI: 6   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


`walk` produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [12]:
X

tensor([[[ 0.6173,  0.4008, -0.4735, -0.7712, -0.3196,  0.1591,  0.9762,
          -0.7801,  0.3353, -0.0524, -0.9032,  0.8997, -0.3554,  0.4499,
           0.1636,  0.5015, -0.1343, -0.5433,  0.3641, -0.4847, -0.9843,
           0.8414, -0.3623,  0.0414,  0.8504, -0.2147, -0.7649,  0.6023,
          -0.3847,  0.2938,  0.9780,  0.6270,  0.9071, -0.7047,  0.9479,
          -0.8610,  0.3039,  0.1934,  0.9549,  0.8189,  0.8731,  0.7472,
          -0.4261,  0.1306, -0.5324,  0.5551, -0.9605, -0.9427, -0.3306,
          -0.8877, -0.6914,  0.9730, -0.7891, -0.4883,  0.4326, -0.9502,
          -0.5412,  0.6529,  0.5179, -0.2092, -0.6117,  0.2445,  0.7106,
          -0.2785, -0.8416, -0.1208, -0.0635, -0.1734,  0.6887,  0.4285,
           0.9015,  0.6880, -0.4168, -0.5558,  0.3563, -0.2213,  0.5438,
          -0.1436, -0.7146, -0.1307,  0.3279, -0.4997, -0.0271,  0.9761,
           0.3781, -0.3052,  0.4314,  0.6846, -0.5006,  0.1158, -0.9094,
           0.7462,  0.3971,  0.2224,  0.0247,  0.72

In [13]:
X.shape

torch.Size([1, 3, 100])

### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a `Hypercube`.

In [14]:
from mhar.polytope_examples import Hypercube

# Create a polytope (Hypercube)
hypercube_sim = Hypercube(10,
                      dtype=torch.float32,
                      device=device
                      )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



Define/Find inner points

In [15]:
x0_sim = ChebyshevCenter(polytope=hypercube_sim, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device=device,
                    solver_precision=np.float32)




Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [16]:
X_sim = walk(polytope=hypercube_sim,
        X0 = x0_sim,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X_sim

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  10   mI: 20   mE: None   z: 100


% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[-1.9521e-01, -4.9613e-01,  9.4348e-01, -3.6091e-01, -2.1067e-01,
          -7.1501e-01,  6.1336e-01,  3.6261e-02, -9.5879e-01,  3.6196e-01,
           8.0419e-01, -6.0833e-01,  1.5020e-01, -5.8299e-01, -5.3218e-01,
           8.4844e-01,  7.6787e-01,  5.5885e-01, -7.2924e-01, -4.7147e-01,
           8.8063e-01, -8.8127e-01,  9.3965e-01,  6.9480e-01,  6.9380e-01,
           5.0325e-01, -6.5507e-01, -4.2953e-01, -6.4428e-01,  8.9482e-01,
           4.1831e-01,  1.9657e-01,  7.5528e-01,  1.0310e-01,  8.9719e-01,
           3.9713e-01,  8.4115e-01, -9.9127e-02, -9.9114e-03,  9.1147e-01,
           9.5645e-01,  6.5732e-01,  2.0378e-01, -3.4835e-01,  3.4385e-01,
          -4.2524e-01, -3.3002e-01, -1.7727e-01,  6.3531e-01, -3.3323e-01,
          -8.0515e-01, -3.0070e-01,  4.8017e-01, -8.3783e-01, -4.3488e-03,
           2.6925e-02, -6.7629e-01, -5.8099e-01, -2.0629e-01, -5.2587e-02,
          -3.8194e-01,  6.7568e-02,  9.5511e-01, -9.9408e-01,  4.3823e-01,
           9.9417e-01, -6

In [17]:
X_sim.shape

torch.Size([1, 10, 100])